<a href="https://colab.research.google.com/github/rts1988/IntelligentTutoringSystem_Experiments/blob/master/Blockorder_clustering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [309]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)



id = "1SaJeqJntPCY0afqmE3uwjwm5HeeXzZ2T"
filename = "Astronomy textbook20211004_signature.csv"
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile(filename) 
import pandas as pd
df = pd.read_csv(filename)


In [1]:
from google.colab import files
uploaded =files.upload()

Saving blockinfo_reviewtextmining_education20220223_signature.csv to blockinfo_reviewtextmining_education20220223_signature.csv


In [2]:
import pandas as pd
df = pd.read_csv(list(uploaded.keys())[0])

In [3]:
def xyeuclidian(x0,y0,x1,y1):
  return ((x1-x0)**2 + (y1-y0)**2)**(1/2)

In [310]:
df.columns

Index(['Unnamed: 0', 'ascender', 'block_xe', 'block_xs', 'block_ye',
       'block_ys', 'blockid', 'blocktype', 'bookname', 'charlist', 'color',
       'descender', 'dir', 'flags', 'font', 'fsize', 'height',
       'imageb_counter', 'imageb_num', 'imageext', 'imageheight', 'imagewidth',
       'imcolorspace', 'line_xe', 'line_xs', 'line_ye', 'line_ys', 'lineid',
       'num_blocks', 'num_lines', 'num_spans', 'numchars', 'page_no',
       'sp_origin_x', 'sp_origin_y', 'span_xe', 'span_xs', 'span_ye',
       'span_ys', 'spanid', 'text', 'text_token', 'textb_counter', 'textb_num',
       'textlen', 'width', 'wmode', 'page_frac', 'num_alphachars',
       'num_numericchars', 'frac_alphachars', 'frac_numericchars',
       'num_specialchars', 'frac_specialchars', 'num_tokens',
       'num_capitalizedtokens', 'frac_capitalizedtokens',
       'num_capitalizedfirstchars', 'frac_capitalizedfirstchars',
       'num_sentenceends', 'block_area', 'line_area', 'span_area',
       'num_stoptokens', 'nu

Getting distance between lines . (instead of blocks)

found it takes too long (~ 5 min and still not finished. )


In [311]:
page_span_info = dict()
span_dist_info = dict()
try:
  df['blockid'] = df['blockid'].apply(eval)
  df['lineid'] = df['lineid'].apply(eval)
  df['spanid'] = df['spanid'].apply(eval)
except:
  pass

In [14]:
df.columns

Index(['Unnamed: 0', 'bookname', 'page_no', 'width', 'height', 'num_blocks',
       'textb_num', 'imageb_num', 'textb_counter', 'imageb_counter', 'blockid',
       'blocktype', 'block_xs', 'block_ys', 'block_xe', 'block_ye',
       'imagewidth', 'imageheight', 'imageext', 'imcolorspace', 'num_lines',
       'lineid', 'line_xs', 'line_ys', 'line_xe', 'line_ye', 'wmode', 'dir',
       'num_spans', 'spanid', 'text', 'text_token', 'textlen', 'ascender',
       'descender', 'color', 'font', 'fsize', 'flags', 'sp_origin_x',
       'sp_origin_y', 'span_xs', 'span_ys', 'span_xe', 'span_ye', 'numchars',
       'charlist', 'page_frac', 'num_alphachars', 'num_numericchars',
       'frac_alphachars', 'frac_numericchars', 'num_specialchars',
       'frac_specialchars', 'num_tokens', 'num_capitalizedtokens',
       'frac_capitalizedtokens', 'num_capitalizedfirstchars',
       'frac_capitalizedfirstchars', 'num_sentenceends', 'block_area',
       'line_area', 'span_area', 'num_stoptokens', 'num_notst

In [312]:
num_pages = df['page_no'].max()
for page in range(int(num_pages)):
  span_info = df.loc[df['page_no']==page,['spanid','text','numchars','ascender','descender','span_xs_trunc','span_ys_trunc','span_xe_trunc','span_ye_trunc']].sort_values(by = ['span_ys_trunc','span_xs_trunc'])
  #print(span_info.shape)
  #span_info = span_info.drop_duplicates()
  closest_span_list = []
  min_horiz_spanid = []
  min_horiz_dist = []
  min_vert_spanid = []
  min_vert_dist = []
  closest_spanid = []
  closest_dir = []
  
  i = 0
  #print()
  #print(block_info[['blockid','block_ys_trunc','block_xs_trunc']])
  span_dist = pd.DataFrame()
  for i,row_i in span_info.iterrows():   
    #fsize = row_i['fsize']
    xs_border = row_i['span_xs_trunc'] - 3#fsize*0.1
    ys_border = row_i['span_ys_trunc'] - 3#fsize*0.1
    xe_border = row_i['span_xe_trunc'] + 3#fsize*
    ye_border = row_i['span_ye_trunc'] + 3#fsize*5
    
    subspan_info = span_info.loc[~((span_info['span_xs_trunc']>xe_border) | (span_info['span_ys_trunc']>ye_border)|(span_info['span_xe_trunc']<xs_border)|(span_info['span_ye_trunc']<ys_border)),:]
    #print('\t',subspan_info.shape[0])

    for j,row_j in subspan_info.iterrows():
      rawdict= dict()
      if i!=j:
        
        rawdict['page'] = page
        rawdict['from_spanid'] = row_i['spanid']
        rawdict['to_spanid'] = row_j['spanid']
        rawdict['horizontal_top_dist'] = xyeuclidian(row_i['span_xe_trunc'],row_i['span_ys_trunc'],row_j['span_xs_trunc'],row_j['span_ys_trunc'])
        rawdict['horizontal_bottom_dist'] = xyeuclidian(row_i['span_xe_trunc'],row_i['span_ye_trunc'],row_j['span_xe_trunc'],row_j['span_ys_trunc'])
        rawdict['horizontal_aligned_dist'] = (rawdict['horizontal_top_dist'] + rawdict['horizontal_bottom_dist'])/2
        rawdict['vertical_left_dist'] = xyeuclidian(row_i['span_xs_trunc'],row_i['span_ye_trunc'],row_j['span_xs_trunc'],row_j['span_ys_trunc'])
        rawdict['vertical_right_dist'] = xyeuclidian(row_i['span_xe_trunc'],row_i['span_ye_trunc'],row_j['span_xe_trunc'],row_j['span_ys_trunc'])
        rawdict['vertical_aligned_dist'] = (rawdict['vertical_left_dist'] + rawdict['vertical_right_dist'])/2
        rawdict['diagonal_dist_itoj'] = xyeuclidian(row_i['span_xe_trunc'],row_i['span_ye_trunc'],row_j['span_xs_trunc'],row_j['span_ys_trunc'])
        rawdict['diagonal_dist_jtoi'] = xyeuclidian(row_j['span_xe_trunc'],row_j['span_ye_trunc'],row_i['span_xs_trunc'],row_i['span_ys_trunc'])
        rawdict['diagonal_dist'] = min(rawdict['diagonal_dist_itoj'],rawdict['diagonal_dist_jtoi'])
      else:
        rawdict['page'] = page
        rawdict['to_spanid'] = row_j['spanid']
        rawdict['from_spanid'] = row_i['spanid']
        rawdict['horizontal_top_dist'] = 0
        rawdict['horizontal_bottom_dist'] = 0
        rawdict['horizontal_aligned_dist'] = 0
        rawdict['vertical_left_dist'] = 0
        rawdict['vertical_right_dist'] = 0
        rawdict['vertical_aligned_dist'] = 0
        rawdict['diagonal_dist_itoj'] = 0
        rawdict['diagonal_dist_jtoi'] = 0
        rawdict['diagonal_dist'] = 0
      rawdict['min_dist'] = min([rawdict['horizontal_top_dist'],rawdict['horizontal_bottom_dist'],
                                  rawdict['horizontal_bottom_dist'],rawdict['horizontal_aligned_dist'],
                                  rawdict['vertical_left_dist'],rawdict['vertical_right_dist'],rawdict['vertical_aligned_dist'],
                                  rawdict['diagonal_dist']])
      span_dist = span_dist.append(rawdict,ignore_index=True)
    notsubspaninfo = span_info.loc[((span_info['span_xs_trunc']>xe_border) | (span_info['span_ys_trunc']>ye_border)|(span_info['span_xe_trunc']<xs_border)|(span_info['span_ye_trunc']<ys_border)),:]
    # for k,row_k in notsubspaninfo.iterrows():
    #   rawdict = dict()
    #   rawdict['page'] = page
    #   rawdict['to_spanid'] = row_j['spanid']
    #   rawdict['from_spanid'] = row_i['spanid']
    #   rawdict['horizontal_top_dist'] = 10000
    #   rawdict['horizontal_bottom_dist'] = 10000
    #   rawdict['horizontal_aligned_dist'] = 10000
    #   rawdict['vertical_left_dist'] = 10000
    #   rawdict['vertical_right_dist'] = 10000
    #   rawdict['vertical_aligned_dist'] = 10000
    #   rawdict['diagonal_dist_itoj'] = 10000
    #   rawdict['diagonal_dist_jtoi'] = 10000
    #   rawdict['diagonal_dist'] = 10000
    #   rawdict['min_dist'] =10000
    #   span_dist = span_dist.append(rawdict,ignore_index=True)
  if page%10==0:
    print(page,'pages done')
  page_span_info[page]=span_info  
  span_dist_info[page] = span_dist


(0, 9)
0 pages done
(0, 9)
(48, 9)
(64, 9)
(55, 9)
(11, 9)
(102, 9)
(106, 9)
(96, 9)
(104, 9)
(102, 9)
(93, 9)
(46, 9)
(40, 9)
(40, 9)
(39, 9)
(4, 9)
(4, 9)
(26, 9)
(59, 9)
(47, 9)
(26, 9)
(38, 9)
(29, 9)
(60, 9)
(55, 9)
(66, 9)
(38, 9)
(79, 9)
(34, 9)
(37, 9)
(45, 9)
(25, 9)
(29, 9)
(36, 9)
(14, 9)
(10, 9)
(30, 9)
(67, 9)
(49, 9)
(36, 9)
(79, 9)
(29, 9)
(38, 9)
(29, 9)
(41, 9)
(40, 9)
(41, 9)
(32, 9)
(25, 9)
(60, 9)
(48, 9)
(34, 9)
(53, 9)
(51, 9)
(38, 9)
(29, 9)
(33, 9)
(32, 9)
(35, 9)
(54, 9)
(59, 9)
(28, 9)
(43, 9)
(41, 9)
(37, 9)
(41, 9)
(45, 9)
(19, 9)
(36, 9)
(44, 9)
(43, 9)
(30, 9)
(56, 9)
(56, 9)
(83, 9)
(58, 9)
(62, 9)
(58, 9)
(9, 9)
(33, 9)
(31, 9)
(47, 9)
(54, 9)
(56, 9)
(70, 9)
(77, 9)
(40, 9)
(66, 9)
(38, 9)
(94, 9)
(33, 9)
(76, 9)
(55, 9)
(38, 9)
(133, 9)
(79, 9)
(40, 9)
(44, 9)
(38, 9)
(46, 9)
100 pages done
(25, 9)
(36, 9)
(48, 9)
(24, 9)
(43, 9)
(8, 9)
(60, 9)
(88, 9)
(66, 9)
(104, 9)
(59, 9)
(63, 9)
(29, 9)
(35, 9)
(47, 9)
(22, 9)
(58, 9)
(31, 9)
(28, 9)
(29, 9)
(55,

KeyboardInterrupt: ignored

In [ ]:
span_dist_info[0]

""


In [7]:
from scipy import sparse
from scipy.sparse import coo_matrix
import numpy as num
from sklearn.cluster import DBSCAN
from sklearn import metrics
from sklearn.preprocessing import StandardScaler
db = DBSCAN(eps=8,min_samples=1,metric = 'precomputed')

In [147]:
# run for each page, at a couple different eps and save binfo_dict to a pickle file, maybe also calculate the order here? or later?
binfo_dict = dict()
for no in range(0,10):
  row = list(span_dist_info[no]['from_spanid'].apply(lambda x:x[2]))
  #print(row)
  col = list(span_dist_info[no]['to_spanid'].apply(lambda x:x[2]))
  data = list(span_dist_info[no]['min_dist'])
  Z = coo_matrix((data, (row, col)), shape=(len(set(row)),len(set(col))))
  dist_matrix = num.empty(Z.shape)
  dist_matrix.fill(10000)
  for i in range(len(row)):
    dist_matrix[row[i],col[i]] = data[i]

  clustering = db.fit(dist_matrix)
  clustering.labels_ 
  labels = pd.DataFrame()
  labels['block_no'] = list(range(0,max(row)+1))
  labels['dbeps8min1'] = clustering.labels_

  binfo = page_span_info[no].copy()
  binfo['block_no'] = binfo['spanid'].apply(lambda x:x[2])
  binfo= pd.merge(left = binfo,right = labels,how = "left",left_on ='block_no',right_on = 'block_no')
  binfo_dict[no] = binfo

In [19]:
binfo_dict[15].sort_values(by = ['dbeps8min1','span_ye_trunc','span_xs_trunc'])[00:60]
#binfo_dict.keys()

,spanid,text,numchars,ascender,descender,span_xs_trunc,span_ys_trunc,span_xe_trunc,span_ye_trunc,block_no,dbeps8min1
4,"(Review Text mining in education.pdf, 15, 0, 0...","Hendricks, R., Curtis, D., & Zeng-Treitler, Q....",121.0,0.958,-0.208,46.0,48.0,457.0,58.0,0,0
5,"(Review Text mining in education.pdf, 15, 0, 0...",E-learn: World conference,25.0,0.947,-0.208,457.0,48.0,550.0,58.0,0,0
9,"(Review Text mining in education.pdf, 15, 0, 1...","on E-learning in corporate, government, health...",71.0,0.947,-0.208,59.0,60.0,317.0,69.0,0,0
6,"(Review Text mining in education.pdf, 15, 0, 1...",(pp. 118,8.0,0.958,-0.208,317.0,59.0,349.0,69.0,0,0
8,"(Review Text mining in education.pdf, 15, 0, 1...",126). Association for the Advancement of Compu...,52.0,0.958,-0.208,354.0,59.0,550.0,69.0,0,0
7,"(Review Text mining in education.pdf, 15, 0, 1...",–,0.0,1.020,-0.333,349.0,59.0,354.0,70.0,0,0
10,"(Review Text mining in education.pdf, 15, 0, 2...",Education (AACE).,16.0,0.958,-0.208,59.0,71.0,126.0,81.0,0,0
11,"(Review Text mining in education.pdf, 15, 0, 3...","Hirokawa, S., & Flanagan, B. (2018). An automa...",129.0,0.958,-0.208,46.0,83.0,502.0,93.0,0,0
12,"(Review Text mining in education.pdf, 15, 0, 3...",International,13.0,0.947,-0.208,502.0,83.0,550.0,93.0,0,0
17,"(Review Text mining in education.pdf, 15, 0, 4...",Journal of Distance Education Technologies,41.0,0.947,-0.208,59.0,95.0,210.0,104.0,0,0


In [148]:
num_pages = df['page_no'].max()
for page in range(int(num_pages)):
  line_info = df.loc[df['page_no']==page,['lineid','line_xs_trunc','line_ys_trunc','line_xe_trunc','line_ye_trunc']].sort_values(by = ['line_ys_trunc','line_xs_trunc'])
  
  line_info = line_info.drop_duplicates()
  print(line_info.shape)
  closest_line_list = []
  min_horiz_lineid = []
  min_horiz_dist = []
  min_vert_lineid = []
  min_vert_dist = []
  closest_lineid = []
  closest_dir = []

  i = 0
  #print()
  #print(block_info[['blockid','block_ys_trunc','block_xs_trunc']])
  line_dist = pd.DataFrame()
  for i,row_i in line_info.iterrows():  
    #fontsize = 

    for j,row_j in line_info.iterrows():
      rawdict= dict()
      if i!=j:
        
        rawdict['page'] = page
        rawdict['from_lineid'] = row_i['lineid']
        rawdict['to_lineid'] = row_j['lineid']
        rawdict['horizontal_top_dist'] = xyeuclidian(row_i['line_xe_trunc'],row_i['line_ys_trunc'],row_j['line_xs_trunc'],row_j['line_ys_trunc'])
        rawdict['horizontal_bottom_dist'] = xyeuclidian(row_i['line_xe_trunc'],row_i['line_ye_trunc'],row_j['line_xe_trunc'],row_j['line_ys_trunc'])
        rawdict['horizontal_aligned_dist'] = (rawdict['horizontal_top_dist'] + rawdict['horizontal_bottom_dist'])/2
        rawdict['vertical_left_dist'] = xyeuclidian(row_i['line_xs_trunc'],row_i['line_ye_trunc'],row_j['line_xs_trunc'],row_j['line_ys_trunc'])
        rawdict['vertical_right_dist'] = xyeuclidian(row_i['line_xe_trunc'],row_i['line_ye_trunc'],row_j['line_xe_trunc'],row_j['line_ys_trunc'])
        rawdict['vertical_aligned_dist'] = (rawdict['vertical_left_dist'] + rawdict['vertical_right_dist'])/2
        rawdict['diagonal_dist_itoj'] = xyeuclidian(row_i['line_xe_trunc'],row_i['line_ye_trunc'],row_j['line_xs_trunc'],row_j['line_ys_trunc'])
        rawdict['diagonal_dist_jtoi'] = xyeuclidian(row_j['line_xe_trunc'],row_j['line_ye_trunc'],row_i['line_xs_trunc'],row_i['line_ys_trunc'])
        rawdict['diagonal_dist'] = min(rawdict['diagonal_dist_itoj'],rawdict['diagonal_dist_jtoi'])
      else:
        rawdict['page'] = page
        rawdict['to_lineid'] = row_j['lineid']
        rawdict['from_lineid'] = row_i['lineid']
        rawdict['horizontal_top_dist'] = 0
        rawdict['horizontal_bottom_dist'] = 0
        rawdict['horizontal_aligned_dist'] = 0
        rawdict['vertical_left_dist'] = 0
        rawdict['vertical_right_dist'] = 0
        rawdict['vertical_aligned_dist'] = 0
        rawdict['diagonal_dist_itoj'] = 0
        rawdict['diagonal_dist_jtoi'] = 0
        rawdict['diagonal_dist'] = 0
      rawdict['min_dist'] = min([rawdict['horizontal_top_dist'],rawdict['horizontal_bottom_dist'],
                                  rawdict['horizontal_bottom_dist'],rawdict['horizontal_aligned_dist'],
                                  rawdict['vertical_left_dist'],rawdict['vertical_right_dist'],rawdict['vertical_aligned_dist'],
                                  rawdict['diagonal_dist']])
      line_dist = line_dist.append(rawdict,ignore_index=True)
  page_line_info[page]= line_info  
  line_dist_info[page] = line_dist


(69, 5)


NameError: ignored

In [ ]:
span_dist_info[0]['from_spanid'][0]

('WinneNesbit2010 The psychology of academic achievement.pdf', 0, 0, 0, 0)

In [ ]:
page_line_info = dict()
line_dist_info = dict()
try:
  df['blockid'] = df['blockid'].apply(eval)
  df['lineid'] = df['lineid'].apply(eval)
  df['spanid'] = df['spanid'].apply(eval)
except:
  pass

In [ ]:
binfo_dict[4].sort_values(by = ['dbeps8min1','span_ys_trunc','span_xs_trunc'])[0:30]

,spanid,text,span_xs_trunc,span_ys_trunc,span_xe_trunc,span_ye_trunc,block_no,dbeps8min1
0,(WinneNesbit2010 The psychology of academic ac...,a,241.0,37.0,244.0,43.0,0,0
1,(WinneNesbit2010 The psychology of academic ac...,Table 1,39.0,38.0,66.0,46.0,0,0
2,(WinneNesbit2010 The psychology of academic ac...,Twenty-five heuristics for promoting learning,74.0,38.0,241.0,46.0,0,0
3,(WinneNesbit2010 The psychology of academic ac...,Contiguity effects,39.0,53.0,99.0,62.0,1,1
4,(WinneNesbit2010 The psychology of academic ac...,Ideas that need to be associated should be pre...,170.0,53.0,453.0,62.0,1,1
5,(WinneNesbit2010 The psychology of academic ac...,Perceptual-motor grounding,39.0,66.0,136.0,74.0,2,1
6,(WinneNesbit2010 The psychology of academic ac...,Concepts benefit from being grounded in percep...,170.0,66.0,481.0,74.0,2,1
7,(WinneNesbit2010 The psychology of academic ac...,stages of learning.,178.0,77.0,239.0,85.0,2,1
8,(WinneNesbit2010 The psychology of academic ac...,Dual code and multimedia effects,39.0,89.0,152.0,97.0,3,2
9,(WinneNesbit2010 The psychology of academic ac...,"Materials presented in verbal, visual, and mul...",170.0,89.0,487.0,97.0,3,2


In [ ]:
num_pages = df['page_no'].max()
try:
  df['blockid'] = df['blockid'].apply(eval)
except:
  pass
page_block_info = dict()
block_dist_info = dict()
for page in range(int(num_pages)):
  
  blockid_list = list(df['blockid'].value_counts().index)
  
  block_info = df.loc[df['page_no']==page,['blockid','block_xs_trunc','block_ys_trunc','block_xe_trunc','block_ye_trunc']].sort_values(by=['block_ys_trunc','block_xs_trunc'])
  block_info = block_info.drop_duplicates()
  block_info = block_info.reset_index(drop=True)
  block_info['first span'] = block_info['blockid'].apply(lambda x: list(df.loc[df['blockid']==x]['text'])[0])
  block_info['last span'] = block_info['blockid'].apply(lambda x: list(df.loc[df['blockid']==x]['text'])[-1])
  closest_block_list = []
  #print(block_info)
  min_horiz_blockid = []
  min_horiz_dist = []
  min_vert_blockid = []
  min_vert_dist = []
  closest_blockid = []
  closest_dir = []
  i = 0
  #print()
  #print(block_info[['blockid','block_ys_trunc','block_xs_trunc']])
  block_dist = pd.DataFrame()
  for i,row_i in block_info.iterrows():   
    for j,row_j in block_info.iterrows():
      rawdict= dict()
      if i!=j:
        
        rawdict['page'] = page
        rawdict['from_blockid'] = row_i['blockid']
        rawdict['to_blockid'] = row_j['blockid']
        rawdict['horizontal_top_dist'] = xyeuclidian(row_i['block_xe_trunc'],row_i['block_ys_trunc'],row_j['block_xs_trunc'],row_j['block_ys_trunc'])
        rawdict['horizontal_bottom_dist'] = xyeuclidian(row_i['block_xe_trunc'],row_i['block_ye_trunc'],row_j['block_xe_trunc'],row_j['block_ys_trunc'])
        rawdict['horizontal_aligned_dist'] = (rawdict['horizontal_top_dist'] + rawdict['horizontal_bottom_dist'])/2
        rawdict['vertical_left_dist'] = xyeuclidian(row_i['block_xs_trunc'],row_i['block_ye_trunc'],row_j['block_xs_trunc'],row_j['block_ys_trunc'])
        rawdict['vertical_right_dist'] = xyeuclidian(row_i['block_xe_trunc'],row_i['block_ye_trunc'],row_j['block_xe_trunc'],row_j['block_ys_trunc'])
        rawdict['vertical_aligned_dist'] = (rawdict['vertical_left_dist'] + rawdict['vertical_right_dist'])/2
        rawdict['diagonal_dist_itoj'] = xyeuclidian(row_i['block_xe_trunc'],row_i['block_ye_trunc'],row_j['block_xs_trunc'],row_j['block_ys_trunc'])
        rawdict['diagonal_dist_jtoi'] = xyeuclidian(row_j['block_xe_trunc'],row_j['block_ye_trunc'],row_i['block_xs_trunc'],row_i['block_ys_trunc'])
        rawdict['diagonal_dist'] = min(rawdict['diagonal_dist_itoj'],rawdict['diagonal_dist_jtoi'])
      else:
        rawdict['page'] = page
        rawdict['to_blockid'] = row_j['blockid']
        rawdict['from_blockid'] = row_i['blockid']
        rawdict['horizontal_top_dist'] = 0
        rawdict['horizontal_bottom_dist'] = 0
        rawdict['horizontal_aligned_dist'] = 0
        rawdict['vertical_left_dist'] = 0
        rawdict['vertical_right_dist'] = 0
        rawdict['vertical_aligned_dist'] = 0
        rawdict['diagonal_dist_itoj'] = 0
        rawdict['diagonal_dist_jtoi'] = 0
        rawdict['diagonal_dist'] = 0
      rawdict['min_dist'] = min([rawdict['horizontal_top_dist'],rawdict['horizontal_bottom_dist'],
                                 rawdict['horizontal_bottom_dist'],rawdict['horizontal_aligned_dist'],
                                 rawdict['vertical_left_dist'],rawdict['vertical_right_dist'],rawdict['vertical_aligned_dist'],
                                 rawdict['diagonal_dist']])
      block_dist = block_dist.append(rawdict,ignore_index=True)
  page_block_info[page]=block_info  
  block_dist_info[page] = block_dist
    
    # getting horizontal closest block dist and 

In [ ]:
# run for each page, at a couple different eps and save binfo_dict to a pickle file, maybe also calculate the order here? or later?
binfo_dict = dict()
for no in range(int(num_pages)):
  row = list(block_dist_info[no]['from_blockid'].apply(lambda x:x[2]))
  #print(row)
  col = list(block_dist_info[no]['to_blockid'].apply(lambda x:x[2]))
  data = list(block_dist_info[no]['min_dist'])
  dist_matrix = coo_matrix((data, (row, col)), shape=(len(set(row)),len(set(col))))
  dist_matrix = dist_matrix.toarray()

  #db = DBSCAN(eps=20,min_samples=1,metric = 'precomputed')
  clustering = db.fit(dist_matrix)
  clustering.labels_
  labels = pd.DataFrame()
  labels['block_no'] = list(range(0,max(row)+1))
  labels['dbeps10min1'] = clustering.labels_

  binfo = page_block_info[no].copy()
  binfo['block_no'] = binfo['blockid'].apply(lambda x:x[2])
  binfo= pd.merge(left = binfo,right = labels,how = "left",left_on ='block_no',right_on = 'block_no')
  binfo_dict[no] = binfo

In [21]:
binfo_dict[0]

,spanid,text,numchars,ascender,descender,span_xs_trunc,span_ys_trunc,span_xe_trunc,span_ye_trunc,block_no,dbeps8min1
0,"(Review Text mining in education.pdf, 0, 21, 0...",Received: 11 March 2019,22.0,0.958,-0.208,44.0,28.0,118.0,36.0,21,14
1,"(Review Text mining in education.pdf, 0, 21, 1...",Revised: 5 June 2019,19.0,0.958,-0.208,133.0,28.0,194.0,36.0,21,14
2,"(Review Text mining in education.pdf, 0, 21, 2...",Accepted: 2 July 2019,20.0,0.958,-0.208,209.0,28.0,273.0,36.0,21,14
3,"(Review Text mining in education.pdf, 0, 22, 0...",DOI: 10.1002/widm.1332,21.0,0.958,-0.208,44.0,41.0,117.0,49.0,22,15
4,"(Review Text mining in education.pdf, 0, 0, 0, 0)",A D V A N C E D R E V I E W,26.0,1.000,-0.208,44.0,66.0,160.0,77.0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
74,"(Review Text mining in education.pdf, 0, 23, 0...",2019;e1332.,11.0,0.958,-0.208,146.0,748.0,183.0,756.0,23,16
75,"(Review Text mining in education.pdf, 0, 23, 1...",wires.wiley.com/dmkd,19.0,0.958,-0.208,270.0,748.0,334.0,756.0,23,16
76,"(Review Text mining in education.pdf, 0, 23, 2...","© 2019 Wiley Periodicals, Inc.",29.0,0.958,-0.208,419.0,748.0,506.0,756.0,23,16
77,"(Review Text mining in education.pdf, 0, 23, 3...",1 of 49,6.0,1.000,-0.208,527.0,748.0,548.0,756.0,23,16


In [24]:
# save binfo dict to a pickle file. 
filename = list(uploaded.keys())[0]
filename = filename.split('.')[0]+'binfodict.pickle'
import pickle
with open(filename, 'wb') as f:
  pickle.dump(binfo_dict, f)
f.close()
from google.colab import files
files.download(filename)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [313]:
df.columns

Index(['Unnamed: 0', 'ascender', 'block_xe', 'block_xs', 'block_ye',
       'block_ys', 'blockid', 'blocktype', 'bookname', 'charlist', 'color',
       'descender', 'dir', 'flags', 'font', 'fsize', 'height',
       'imageb_counter', 'imageb_num', 'imageext', 'imageheight', 'imagewidth',
       'imcolorspace', 'line_xe', 'line_xs', 'line_ye', 'line_ys', 'lineid',
       'num_blocks', 'num_lines', 'num_spans', 'numchars', 'page_no',
       'sp_origin_x', 'sp_origin_y', 'span_xe', 'span_xs', 'span_ye',
       'span_ys', 'spanid', 'text', 'text_token', 'textb_counter', 'textb_num',
       'textlen', 'width', 'wmode', 'page_frac', 'num_alphachars',
       'num_numericchars', 'frac_alphachars', 'frac_numericchars',
       'num_specialchars', 'frac_specialchars', 'num_tokens',
       'num_capitalizedtokens', 'frac_capitalizedtokens',
       'num_capitalizedfirstchars', 'frac_capitalizedfirstchars',
       'num_sentenceends', 'block_area', 'line_area', 'span_area',
       'num_stoptokens', 'nu

In [314]:
df['frac_stoptokens_discrete'] = pd.qcut(df['frac_stoptokens'],2,duplicates='drop')
df['frac_notstoptokens_discrete'] = pd.qcut(df['frac_notstoptokens'],2,duplicates='drop')
df['frac_alphachars_discrete'] = pd.qcut(df['frac_alphachars'],2,duplicates='drop')
df['frac_numericchars_discrete'] = pd.qcut(df['frac_numericchars'],2,duplicates='drop')
df['frac_specialchars_discrete'] = pd.qcut(df['frac_specialchars'],2,duplicates='drop')
df['num_tokens_discrete'] = pd.qcut(df['num_tokens'],2,duplicates='drop')
df['frac_capitalizedtokens_discrete'] = pd.qcut(df['frac_capitalizedtokens'],2,duplicates='drop')
df['frac_capitalizedfirstchars_discrete'] = pd.qcut(df['frac_capitalizedfirstchars'],2,duplicates='drop')
df['num_sentenceends_discrete'] = pd.qcut(df['num_sentenceends'],2,duplicates='drop')
df['block_ys_trunc_discrete'] = pd.qcut(df['block_ys_trunc'],4,duplicates='drop')
df['block_xs_trunc_discrete'] = pd.qcut(df['block_xs_trunc'],4,duplicates='drop')
df['block_xe_trunc_discrete'] = pd.qcut(df['block_xe_trunc'],4,duplicates='drop')
df['block_ye_trunc_discrete'] = pd.qcut(df['block_ye_trunc'],4,duplicates='drop')

In [358]:
# what is the font size for which the median stop characters is highest?
import numpy as num
textclasses = df.groupby(['fsize','num_tokens_discrete','block_xs_trunc_discrete','font']).agg(count = ('fsize',len),median_fracstoptokens= ('frac_stoptokens',num.nanmedian),median_caps = ('frac_capitalizedtokens',num.nanmedian),median_specialchars= ('frac_specialchars',num.nanmedian), median_sentenceends = ('num_sentenceends',num.nanmedian),min_ys = ('block_ys_trunc',min),max_ys = ('block_ys_trunc',max),min_xs = ('block_xs_trunc',min),max_xs=('block_xs_trunc',max),min_page_no = ('page_no',min),max_page_no = ('page_no',max),textlist = ('text',list),fontlist = ('font',frozenset),blocklist = ('blockid',frozenset)).sort_values(by=['median_fracstoptokens','count'],ascending=[False]*2)
textclasses['fsize'] = textclasses.index.get_level_values(0)
textclasses['num_tokens_discrete'] = textclasses.index.get_level_values(1)
#textclasses.head(20)
textclasses['page_range']  = textclasses['max_page_no'] - textclasses['min_page_no']

textclasses.sort_values(by=['count','page_range'],ascending=[False]*2).head(50)


count  ...  page_range
fsize  num_tokens_discrete block_xs_trunc_discrete font                             ...            
9.000  (7.0, 46.0]         (59.999, 72.0]          NotoSans                20466.0  ...      1184.0
       (-0.001, 7.0]       (59.999, 72.0]          NotoSans                 7355.0  ...      1193.0
                           (80.0, 532.0]           NotoSans                 4994.0  ...      1198.0
                           (59.999, 72.0]          NotoSans-Bold            4322.0  ...      1182.0
       (7.0, 46.0]         (80.0, 532.0]           NotoSans                 3923.0  ...      1189.0
       (-0.001, 7.0]       (72.0, 80.0]            NotoSans                 3643.0  ...      1156.0
6.923  (7.0, 46.0]         (59.999, 72.0]          NotoSans                 2287.0  ...      1172.0
7.500  (-0.001, 7.0]       (59.999, 72.0]          Dosis-Regular            1746.0  ...      1193.0
9.000  (-0.001, 7.0]       (59.999, 72.0]          NotoSans-Italic          1587.0  ...      1192.0
                           (80.0, 532.0]           NotoSans-Bold            1453.0  ...      1190.0
7.500  (7.0, 46.0]         (59.999, 72.0]          Dosis-Regular            1182.0  ...      1198.0
10.000 (-0.001, 7.0]       (80.0, 532.0]           STIXGeneral-Regular       924.0  ...      1112.0
11.700 (-0.001, 7.0]       (59.999, 72.0]          NotoSans-Bold             669.0  ...      1192.0
6.750  (-0.001, 7.0]       (59.999, 72.0]          NotoSans                  508.0  ...      1149.0
8.000  (-0.001, 7.0]       (80.0, 532.0]           STIXGeneral-Regular       491.0  ...      1078.0
6.923  (-0.001, 7.0]       (59.999, 72.0]          NotoSans-Bold             470.0  ...      1176.0
10.000 (-0.001, 7.0]       (80.0, 532.0]           STIXGeneral-Italic        328.0  ...      1057.0
6.923  (-0.001, 7.0]       (59.999, 72.0]          NotoSans                  325.0  ...      1167.0
9.000  (-0.001, 7.0]       (80.0, 532.0]           NotoSans-Italic           289.0  ...      1161.0
6.923  (7.0, 46.0]         (59.999, 72.0]          NotoSans-Bold             287.0  ...      1127.0
6.750  (-0.001, 7.0]       (72.0, 80.0]            NotoSans                  262.0  ...      1021.0
11.700 (7.0, 46.0]         (80.0, 532.0]           Dosis-SemiBold            241.0  ...      1115.0
9.000  (7.0, 46.0]         (59.999, 72.0]          NotoSans-Bold             212.0  ...      1165.0
13.500 (-0.001, 7.0]       (59.999, 72.0]          Dosis-Bold                207.0  ...      1186.0
                           (80.0, 532.0]           Dosis-Bold                200.0  ...      1144.0
6.750  (-0.001, 7.0]       (80.0, 532.0]           NotoSans                  194.0  ...      1151.0
9.000  (-0.001, 7.0]       (72.0, 80.0]            NotoSans-Bold             182.0  ...      1118.0
6.923  (-0.001, 7.0]       (72.0, 80.0]            Dosis-SemiBold            155.0  ...      1110.0
9.000  (7.0, 46.0]         (72.0, 80.0]            NotoSans                  150.0  ...      1099.0
13.500 (-0.001, 7.0]       (72.0, 80.0]            Dosis-SemiBold            140.0  ...      1110.0
6.923  (7.0, 46.0]         (80.0, 532.0]           NotoSans                   86.0  ...      1001.0
12.000 (-0.001, 7.0]       (80.0, 532.0]           NotoSans-Bold              81.0  ...         5.0
9.000  (7.0, 46.0]         (80.0, 532.0]           NotoSans-Bold              79.0  ...      1169.0
7.159  (-0.001, 7.0]       (80.0, 532.0]           STIXSizeOneSym-Regular     78.0  ...       738.0
6.923  (-0.001, 7.0]       (59.999, 72.0]          NotoSans-Italic            77.0  ...      1078.0
9.000  (-0.001, 7.0]       (59.999, 72.0]          NotoSans-BoldItalic        73.0  ...      1074.0
11.700 (-0.001, 7.0]       (80.0, 532.0]           NotoSans-Bold              65.0  ...      1158.0
5.746  (-0.001, 7.0]       (80.0, 532.0]           NotoSans                   53.0  ...      1156.0
7.500  (-0.001, 7.0]       (59.999, 72.0]          Dosis-Bold                

In [ ]:
for i,row in textclasses.loc[textclasses['count']>1000,:].iterrows():
  print(row[['count', 'median_fracstoptokens',	'median_specialchars',	'median_sentenceends',	'min_ys',	'max_ys',	'min_xs',	'max_xs',	'min_page_no',	'max_page_no']])
  for r in row['textlist'][0:20]:
    print(r)

In [350]:
# df1 = df.copy()
# df = df.loc[df['page_no']<11,:].copy()
collist = ['fsize','num_tokens_discrete','frac_stoptokens_discrete','frac_specialchars_discrete']
# categorizing text type by the font used
#textclasses = df[['fsize','font','ascender','descender','frac_stoptokens_discrete','frac_specialchars_discrete']].value_counts()
textclasses = df[collist].value_counts()
textclasses

fsize      num_tokens_discrete  frac_stoptokens_discrete  frac_specialchars_discrete
9.000000   (7.0, 46.0]          (0.222, 1.0]              (-0.001, 1.0]                 22683
           (-0.001, 7.0]        (-0.001, 0.222]           (-0.001, 1.0]                 19974
                                (0.222, 1.0]              (-0.001, 1.0]                  3997
           (7.0, 46.0]          (-0.001, 0.222]           (-0.001, 1.0]                  2177
6.923000   (7.0, 46.0]          (0.222, 1.0]              (-0.001, 1.0]                  2086
                                                                                        ...  
12.960000  (-0.001, 7.0]        (-0.001, 0.222]           (-0.001, 1.0]                     1
7.920000   (-0.001, 7.0]        (0.222, 1.0]              (-0.001, 1.0]                     1
14.189433  (-0.001, 7.0]        (-0.001, 0.222]           (-0.001, 1.0]                     1
15.840000  (-0.001, 7.0]        (-0.001, 0.222]           (-0.001, 1.

In [351]:
len(textclasses)

72

In [352]:
classid = 0
print(textclasses.index[classid])
textclasssub = df.copy()

for col,i in zip(collist,range(len(collist))):
  textclasssub = textclasssub.loc[textclasssub[col]==textclasses.index[classid][i],:]
page_range = pd.Series(textclasssub['page_no'].value_counts().index).describe()
print(page_range)
text_sample = textclasssub[['page_no','text']]
text_sample
#text_sample.loc[text_sample['page_no']==0]

(9.0, Interval(7.0, 46.0, closed='right'), Interval(0.222, 1.0, closed='right'), Interval(-0.001, 1.0, closed='right'))
count    1138.000000
mean      591.778559
std       335.829576
min         8.000000
25%       302.250000
50%       591.500000
75%       882.750000
max      1196.000000
dtype: float64


,page_no,text
429,8.0,The Origin and Fate of Comets and Related Obj...
468,8.0,Sources of Sunshine: Thermal and Gravitationa...
471,8.0,"Mass, Energy, and the Theory of Relativity"
553,9.0,The H–R Diagram and the Study of Stellar Evol...
571,9.0,Evolution from the Main Sequence to Red Giants
...,...,...
56683,1196.0,apps that allow you to simply hold your phone ...
56686,1196.0,is a sky chart that turns inside a round fram...
56687,1196.0,on any date and time of the year. You can buy ...
56688,1196.0,you can construct your own from templates at t...


In [102]:
texttypedict = dict()
texttypedict[0] = 'main_text'
texttypedict[1] = 'table_data'
texttypedict[2] = 'main_text'
texttypedict[3] = 'headings'
texttypedict[4] = 'headers'
texttypedict[5] = 'table_title'
texttypedict[12] = 'headings'
texttypedict[13] = 'table_data'
texttypedict[14] = 'table_nos'
texttypedict[15] = 'main_text_sp'
texttypedict[16] = 'table_head'
texttypedict[19] = 'main_text_sp'
texttypedict[20] = 'table_head'
texttypedict[25] = 'headings'
texttypedict[30] = 'title'




# texttypedict[1] = 'table_main'
# texttypedict[2] = 'main_text'
# texttypedict[3] = 'headings'
# texttypedict[4] = 'header'
# texttypedict[5] = 'header'
# texttypedict[6] = 'main_text'
# texttypedict[7] = 'main_text'
# texttypedict[8] = 'table_head'
# texttypedict[15] = 'headings'
# texttypedict[17] = 'main_text'
# texttypedict[18] = 'captions'



,Unnamed: 0,bookname,page_no,width,height,num_blocks,textb_num,imageb_num,textb_counter,imageb_counter,blockid,blocktype,block_xs,block_ys,block_xe,block_ye,imagewidth,imageheight,imageext,imcolorspace,num_lines,lineid,line_xs,line_ys,line_xe,line_ye,wmode,dir,num_spans,spanid,text,text_token,textlen,ascender,descender,color,font,fsize,flags,sp_origin_x,...,frac_numericchars,num_specialchars,frac_specialchars,num_tokens,num_capitalizedtokens,frac_capitalizedtokens,num_capitalizedfirstchars,frac_capitalizedfirstchars,num_sentenceends,block_area,line_area,span_area,num_stoptokens,num_notstoptokens,frac_stoptokens,frac_notstoptokens,fsize_common_rank,fsize_rank,frac_alphachars_cut,block_xs_trunc,block_ys_trunc,block_xe_trunc,block_ye_trunc,line_xs_trunc,line_ys_trunc,line_xe_trunc,line_ye_trunc,span_xs_trunc,span_ys_trunc,span_xe_trunc,span_ye_trunc,frac_stoptokens_discrete,frac_notstoptokens_discrete,frac_alphachars_discrete,frac_numericchars_discrete,frac_specialchars_discrete,num_tokens_discrete,frac_capitalizedtokens_discrete,frac_capitalizedfirstchars_discrete,num_sentenceends_discrete


In [286]:
textclasses.loc[].index

KeyError: ignored